# Table of Contents:

## Pick up the threads


## Data Ingestion and Collector Layer
**1. Apache flume: Introduction**

**2. Flume Components and Sources**

**3. Hands-on tutorial**
    -3.1 Source type: netcat Source
    -3.2 Source type: Exec Source

**4. Quiz**

## Data Storage Layer

**5. HDFS : Hadoop Distributed File System**

**6. GlusterFS: Dependable Distributed File System**

**7. AWS S3**

## Pick up the threads

We have discussed various aspects of Big Data in the introduction concept. Big Data Architectures are composed of various layers and we'll be covering each layer as a concept. In each layer we will see the tools involved to serve the layer functionality. Let's start with Data Ingestion and Collector layer.

# 1. Apache Flume

## Introduction
Data ingestion and collection at one place is required before giving it out for processing. Apache Flume is one tool which help in ingestion and collection. The broad idea of introducing Apache Flume is to show why ingestion and collection is required when you have multiple sources generating data.

**Apache Flume** is a distributed, reliable, and available system for efficiently collecting, aggregating and moving large amounts of log data from many different sources to a centralized data store. 


## Flume Architecture

<img src = "images/Flume-Architecture.png">

## 2. Flume Components

A Flume data flow is made up of five main components: **Events, Sources, Channels, Sinks, and Agents**:

**Events** -  An event is the basic unit of data that is moved using Flume. It is similar to a message and is generally small. It is made up of headers and a byte-array body.

**Sources** - The source receives the event from some external entity and stores it in a channel. The source must understand the type of event that is sent to it.

**Channels** -  A channel is an internal passive store with certain specific characteristics. An in-memory channel, for example, can move events very quickly, but does not provide persistence. A file-based channel provides persistence. 

**Sinks** - The sink removes the event from the channel and forwards it to either to a destination, like HDFS, or to another agent/dataflow. The sink must output an event that is appropriate to the destination.

**Agents** - An agent is the container for a Flume data flow. It is any physical JVM running Flume. An agent must contain at least one source, channel, and sink, but the same agent can run multiple sources, sinks, and channels. A particular data flow path is set up through the configuration process.

Note: A source stores an event in the channel where it stays until it is consumed by a sink. This temporary storage lets source and sink run asynchronously.


## Types of Sources

<img src = "images/flume-type-sources.jpg">

## 3. Setup an Agent

- Flume agent configuration is stored in a local configuration file. This is a text file that follows the Java properties file format. 
- Configurations for one or more agents can be specified in the same configuration file. 
- The configuration file includes properties of each source, sink and channel in an agent and how they are wired together to form data flows.
- Each component (source, sink or channel) in the flow has a name, type, and set of properties that are specific to the type and instantiation. 
- All attributes of a component needs to be set in the properties file of the hosting Flume agent.
- Each configuration file listing of the names of each of the sources, sinks and channels in the agent, and then specifying the connecting channel for each sink and source.

**Let's write a configuration file to see how it works**

Open up your favourite editor to write a `.conf`, let's name it `example1.conf`. 

We'll take up Source for our agent as **netcat** and sink to be **logger** so that we can see data coming in real time.

### 3.1 netcat source:

Working with netcat source will require basic understanding of ports in networking. Let's quickly revise what are ports and why are they useful?

IP address has been used to reach the destination computer. But now, how to identify the correct socket to reach the destined process? For this, the sockets are assigned special numbers called “Port Numbers” , which help to identify the exact sender and receiver processes (applications/ programs). So, along with IP address, port numbers are also provided and both of these work together to locate the destination computer and then the destined process.

**A netcat-like source opens a specified port and listens for data and turns each line of text into an event.** The expectation is that the supplied data is newline separated text. Each line of text is turned into a Flume event and sent via the connected channel.

Below is the configuration file required to setup this netcat agent.



```bash
# firstAgent.conf: A single-node Flume configuration

# Name the components on this agent
myAgent.sources = r1
myAgent.sinks = k1
myAgent.channels = c1

# Describe/configure the source
myAgent.sources.r1.type = netcat
myAgent.sources.r1.bind = localhost
myAgent.sources.r1.port = 55555

# Describe the sink
myAgent.sinks.k1.type = logger

# Use a channel which buffers events in memory
myAgent.channels.c1.type = memory
myAgent.channels.c1.capacity = 1000
myAgent.channels.c1.transactionCapacity = 100

# Bind the source and sink to the channel
myAgent.sources.r1.channels = c1
myAgent.sinks.k1.channel = c1
```


Hang on! Let's have a section wise look to the instructions given in this conf file


### First Section - Naming the components

```bash
# Name the components on this agent
myAgent.sources = r1
myAgent.sinks = k1
myAgent.channels = c1
```

- The first qualifier in the above three lines is the agent name. We can assign any name of our choice to the agent. Here we have assigned a name to the agent as `myAgent`.

- Second qualifier denotes any component among sources, channels and sinks. Here the keywords (sources, channels, sinks) used for second qualifier are fixed and these can’t be replaced with any other names to refer the same components.

- The value assigned on each line at the right side is the name given to each component for this agent. Here we have named, source as `r1`, sink as `k1` and the channel as `c1`. Names can be any strings without space in between. Strings seperated with space are considered to be multiple entries.



### Second Section - Configure the source 

```bash
#Describe/configure the source
myAgent.sources.r1.type = netcat
myAgent.sources.r1.bind = localhost
myAgent.sources.r1.port = 55555

```

- Second section specify the configuration for the source. 
- First qualifier - Agent name.
- Second qualifier - Reserved keyword for sources. 
- Third qualifier - Source name given in the first section.
- Fourth qualifier specifies additional properties of source such as type, port and bind.
- Right side values are specific values for source. We can specify as many properties as available for the source.
- Here we are binding localhost:55555 with the netcat source.



### Third section  - Describe the sink

```bash
myAgent.sinks.k1.type = logger
```

**Logger sink**:

Logs event at INFO level. Typically useful for testing/debugging purpose. Logging the raw stream of data flowing through the ingest pipeline is not desired behaviour in many production environments because this may result in leaking sensitive data or security related configurations, such as secret keys, to Flume log files.

**Relate with the logger module in python**
Python has a built-in module logging which allows writing status messages to a file or any other output streams. The file can contain the information on which part of the code is executed and what problems have been arisen. This logging also has various levels like Debug, Infor, Warning etc. Similarly we will write the data to a port using netcat and those will then go to logger sink.

### Fourth Section - Channel specification

```bash
# Use a channel which buffers events in memory
myAgent.channels.c1.type = memory
myAgent.channels.c1.capacity = 1000
myAgent.channels.c1.transactionCapacity = 100
```

**Memory Channel:**

- Channels are the repositories where the events are staged on a agent
- The events are stored in an in-memory queue with configurable max size.
- Capacity refers to the maximum number of events stored in the channel.
- While the transaction capacity refers to the maximum number of events the channel will take from a source or give to a sink per transaction.



## Starting an agent
An agent is started using a shell script called flume-ng which is located in the bin directory of the Flume distribution. You need to specify the agent name, the config directory, and the config file on the command line:


` flume-ng agent --conf conf --conf-file example1.conf --name myAgent -Dflume.root.logger=INFO,console`


**Start producing some data at source**

Since we have specified netcat as our source and binded 55555 port so let's post some data on this port:

Open up another terminal and run the below command: 

`telnet localhost 55555`

Next, you'll see a prompt to post some data at this port. Similar to this:

<img src="images/netcat.png">


Let's switch to the terminal where we started our Flume agent and see how our posted data looks like. You'll see similar output

<img src = "images/myAgent.png">

**Found something strange?**

If you notice, only first 16 characters of your message is being displayed in the flume agent console. Does this mean that your rest message is lost?

Answer to such behaviour is:

The default logger sink truncates the body content to 16 bytes so only first 16 characters are visible on the console.

## Flow chart describing a transaction event

<img src = "images/transact.png">

Let's do one more exercise with some other source.

## 3.2 Source type: Exec Source

**Exec Source:** 

- Exec source runs a given Unix command on start-up and expects that process to continuously produce data on standard out (stderr is simply discarded, unless property logStdErr is set to true). 

- If the process exits for any reason, the source also exits and will produce no further data. This means configurations such as cat [named pipe] or tail -F [file] are going to produce the desired results where as date will probably not - the former two commands produce streams of data where as the latter produces a single event and exits.



Open up a new file, let's name it `example2.conf`. Here our source will be **exec** and we'll write the ingested data to a file instead of logging it into the console.

```bash
### Agent2 Configuration - Exec Source, File Roll Sink & File Channel ###

# Name the components on this agent 
Agent2.sources = exec-source  
Agent2.channels = file-channel
Agent2.sinks = file-sink

# Describe/configure Source
Agent2.sources.exec-source.type = exec
Agent2.sources.exec-source.command = tail -F /var/log/syslog

# Describe the sink
Agent2.sinks.file-sink.type = FILE_ROLL
Agent2.sinks.file-sink.sink.directory = /home/prakshi/Prakshi/GreyAtom/flume_examples/op_files
Agent2.sinks.file-sink.sink.rollInterval = 0

# Use a channel which buffers events in file
Agent2.channels.file-channel.type = file
Agent2.channels.file-channel.checkpointDir = /home/prakshi/Prakshi/GreyAtom/flume_examples/checkpoint
Agent2.channels.file-channel.dataDirs = /home/prakshi/Prakshi/GreyAtom/flume_examples/data/

# Bind the source and sink to the channel
Agent2.sources.exec-source.channels = file-channel
Agent2.sinks.file-sink.channel = file-channel
```

**Note:** Make sure your `checkpontDir` and `dataDirs` are already created and accessible by the flume agent. Flume can only make files to write output but cannot create directory for you.

**What does the `tail` command do?**

The tail command is used to display contents of a file from the end. 

## Channel type: file-channel

- Channels are the repositories where the events are staged on a agent
- FileChannel is a persistent Flume channel that supports writing to multiple disks in parallel and encryption.
- It writes out all events to disk and thus does not lose data on process or machine shutdown or crash.
- The File Channel is designed to be used in situations where data durability is required and data loss cannot be tolerated.
- The checkpoint reflects the exact state of the channel at the instant the checkpoint was written out.
- `checkpointDir` - The directory where checkpoint file will be stored
- On restart, the channel loads the last checkpoint written out and only replays the puts and takes that happened after this checkpoint, allowing the channel to start up quickly and be ready for normal operation. 
- `dataDirs` - Comma separated list of directories for storing log files. Using multiple directories on separate disks can improve file channel peformance


Flume is a transactional system and multiple events can be either Put or Taken in a single transaction. The batch size can be used to control throughput. 

A Flume transaction consists of either Puts or Takes, but not both, and either a commit or a rollback. Each transaction implements both a Put and Take method. Sources do Puts onto the channel and Sinks do Takes from the channel.



## Start the Flume agent

Start the flume agent for the second configuration file we prepared just now by giving the following command:

`flume-ng agent --conf conf --conf-file example2.conf --name Agent2 -Dflume.root.logger=INFO,console`

Go to the location of `dataDirs` you specified in the configuration file. There you'll see a file created by the flume agent which contains the output of `tail` command which was originally written into the flume channel and passed to the sink. For example my file looks like as shown below and will contain system logs:

<img src = "images/tailOutput.png">

There are many more types of Sources, channels and sink available which you can try in a similar manner. 

If you want to explore more sources then you can refer to the official documentation of Apache Flume which serve as a guide to setup an agent: [Apache Flume](https://flume.apache.org/FlumeUserGuide.html)

# 4. Quiz

**Q.1 What are the important steps in the configuration?**

A. Every Source must have atleast one channel

B. Every Sink must have only one channel

C. Every Components must have a specific type

D. All of the above

Answer - D

**Q.2 Which among the following is core components of Flume?**

A. Event

B. Source

C. Sink

D. All of the above

Answer -D

**Q.3 What does event body in Flume represent?**

A. Key/value pair of an event

B. Metadata of event

C. Content of an event

D. None of these

Answer - C

**Q.4 What are the different channel types in Flume?**

A. Memory Channel

B. JDBC Channel

C. File Channel

D. All of these

Answer - D



# Big Data Storage Layer

In building up any big data pipeline, very first task is to decide, **where your data will be stored?**.
an important point to consider is to keep data in the right place based on usage. An ideal data storage option is the one where data security rules can be applied, can scale as much as required, cost efficient, easy to maintain and Faster response times.

We have relational Databases that were a successful place to store our data over the years. But now we have different types of data coming into our applications from different sources, so look for a storage solution that covers all the above mentioned points. Keeping in mind the role of a Data Scientist, I'll here introduce three popular Big Data Storage tools at a very basic level:


## 5. HDFS : Hadoop Distributed File System

We have introduced HDFS while explaining the Big Data Programming models, let's have a look into how read and write operations happen in the HDFS.

**Name node** - Manages the file system metadata. Metadata includes administrative information about creation time, access properties, and so on related to the data stored.

**Data node** - Manage data storage on individual compute nodes.




**-->  Read operation:**

    - HDFS client provides command line interface to communicate with that distributed file system. So, there is no need to write any code in program languages to access data. 
    
    - First, you request name node to get information about file blocks locations. These blocks are distributed over different machines, but all of this complexity is hidden behind HDFS API.
    
    - The data is served from the closest machine. Closeness here intend to  physical distance and unpredictable system load across the cluster. 
        - distance(/d1/r1/n1, /d1/r1/n1) = 0 (processes on the same node)
        - distance(/d1/r1/n1, /d1/r1/n2) = 2 (different nodes on the same rack)
        - distance(/d1/r1/n1, /d1/r2/n3) = 4 (nodes on different racks in the same data center)
        - distance(/d1/r1/n1, /d2/r3/n4) = 6 (nodes in different data centers)
    
<img src="images/hdfs_distance.png">


**-->  Write operation:**

    -  When you write a block of data into HDFS, Hadoop distributes replicas over the storage. The first replica is usually located on the same node if you write data from a DataNode machine. Otherwise, the first DataNode to put replica is chosen by random.
    
    - The second replica is usually placed in a different rack. You find if this racks goes down, for instance because of power supply problems, then you will still be able to access data from another rack. 
    
    - The third replica is located on a different machine in the same rack as the second replica. Further replicas applies on the random nodes in the cluster although the system tries to avoid placing too many replicas on the same rack. 
    
    - Same is depicted in diagram below where there is one NameNode, and multiple DataNodes (servers). b1, b2,  indicates data blocks.
    
<img src = "images/hdfs.png"> 


So this is was a high level view on uderstanding read and write operations inside hdfs. 

## 6. GlusterFS: Dependable Distributed File System

Good storage solution must provide elasticity in both storage and performance without affecting active operations.

- Scale-out storage systems based on GlusterFS are suitable for unstructured data such as documents, images, audio and video files, and log files. GlusterFS is a Open Source scalable network filesystem.

- Using this, we can create large, distributed storage solutions for media streaming, data analysis, and other data- and bandwidth-intensive tasks.

- You can deploy GlusterFS with the help of commodity hardware servers.

- Linear scaling of performance and storage capacity.

- Scale storage size up to several petabytes, which can be accessed by thousands for servers.

- Keeping in mind the scope of this course, we will not go deep into this tool.

## 7. AWS S3

Amazon Simple Storage Service (Amazon S3) is **object storage** with a simple web service interface to store and retrieve any amount of data from anywhere on the internet.

It is designed to deliver 99.999999999% durability, and scale past trillions of objects worldwide.

Customers use S3 as primary storage for cloud-native applications; as a bulk repository, or "data lake," for analytics; as a target for backup & recovery and disaster recovery; and with serverless computing.


It is scalable, just keep pushing data and the s3 bucket will keep scaling up. Data security is provided through various encryption provided with the service.

Once data is stored on Amazon S3, it can be automatically tiered into lower cost, longer-term cloud storage classes like S3 Standard - Infrequent Access and Amazon Glacier for archiving.


### **Case Studies**

<img src = "images/s3_usecases.png">


**You can read more on these case studies on the link https://aws.amazon.com/s3**